In [ ]:
# essential imports

import numpy as np
from numpy.fft import ifft2, fftshift

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, CenteredNorm

from scipy.constants import c

import torch
from pytorch_finufft.functional import finufft_type1 as nufft2

import h5py

device = "cuda:0"  # use GPU, change to "cpu" if no GPU available

In [ ]:
# define the preset values and import model image

img = h5py.File("test_model.h5", "r")["model"][()]  # import the test model
img_size = img.shape[0]  # 60
fov = np.deg2rad(6000 / 3600)  # fov / 3600 to convert from asec to deg
freq = 230e9
wavelength = c / freq

delta_uv = fov ** (-1)

In [ ]:
uu = (
    np.arange(
        start=(-img_size / 2) * delta_uv,
        stop=(img_size / 2) * delta_uv,
        step=delta_uv,
        dtype=np.float128,
    )
).astype(np.float64)

vv = np.copy(uu)

uv_grid = np.meshgrid(uu, vv)

bins = np.arange(
    start=-(img_size / 2 + 1 / 2) * delta_uv,
    stop=(img_size / 2 + 1 / 2) * delta_uv,
    step=delta_uv,
    dtype=np.float128,
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
for b in bins:
    ax.axvline(x=b, color="black")
    ax.axhline(y=b, color="black")

ax.scatter(uv_grid[0], uv_grid[1], s=10, color="royalblue")
ax.set_xlabel("$u$ in $\\lambda$")
ax.set_ylabel("$v$ in $\\lambda$")

None

In [ ]:
def create_rd_grid(fov, img_size, dec):
    res = fov / img_size
    r = torch.from_numpy(
        np.arange(
            start=-(img_size / 2) * res,
            stop=(img_size / 2) * res,
            step=res,
            dtype=np.float128,
        ).astype(np.float64)
    ).to(device)
    d = r + dec

    R, _ = torch.meshgrid((r, r), indexing="ij")
    _, D = torch.meshgrid((d, d), indexing="ij")
    rd_grid = torch.cat([R[..., None], D[..., None]], dim=2)

    return rd_grid


def create_lm_grid(fov, img_size, dec):

    dec = np.deg2rad(dec).astype(np.float128)

    rd = (
        create_rd_grid(fov=fov, img_size=img_size, dec=dec)
        .cpu()
        .numpy()
        .astype(np.float128)
    )

    lm_grid = np.zeros(rd.shape, dtype=np.float128)
    lm_grid[..., 0] = np.cos(rd[..., 1]) * np.sin(rd[..., 0])
    lm_grid[..., 1] = np.sin(rd[..., 1]) * np.cos(dec) - np.cos(rd[..., 1]) * np.sin(
        dec
    ) * np.cos(rd[..., 0])

    return torch.from_numpy(lm_grid.astype(np.float64)).to(device)

In [ ]:
lm_grid = create_lm_grid(fov=np.deg2rad(90), img_size=img_size, dec=0)

l = lm_grid[..., 0, 0].cpu().numpy()
m = lm_grid[..., 1, 0].cpu().numpy()

In [ ]:
fig, ax = plt.subplots(
    1,
    1,
    layout="constrained",
    figsize=(7, 7),
)
ax.scatter(
    lm_grid[..., 0].cpu().numpy(), lm_grid[..., 1].cpu().numpy(), s=7, color="purple"
)
ax.set_ylabel("$m$")
ax.set_xlabel("$l$")

None

In [ ]:
lm_grid = create_lm_grid(fov=fov, img_size=img_size, dec=0)

In [ ]:
x = 2 * torch.pi * lm_grid[..., 0].flatten() / fov
y = 2 * torch.pi * lm_grid[..., 1].flatten() / fov

img_flat = torch.tensor(img, dtype=torch.complex128)
img_flat = img_flat.to(device).flatten()  # send to GPU and transform image to vector

stokes_i = (
    nufft2(
        points=torch.vstack([x, y]),
        values=img_flat,
        output_shape=(img_size, img_size),
        isign=-1,
        eps=1e-15,
    )
    .cpu()
    .numpy()
)  # compute nonuniform FFT

In [ ]:
real = stokes_i.real.T
imag = stokes_i.imag.T

samps = np.array(
    [
        np.append(-uv_grid[0].ravel(), uv_grid[0].ravel()),
        np.append(-uv_grid[1].ravel(), uv_grid[1].ravel()),
        np.append(real.ravel(), real.ravel()),
        np.append(-imag.ravel(), imag.ravel()),
    ]
)

mask, *_ = np.histogram2d(samps[0], samps[1], bins=[bins, bins], density=False)
mask[mask == 0] = 1

mask_real, x_edges, y_edges = np.histogram2d(
    samps[0], samps[1], bins=[bins, bins], weights=samps[2], density=False
)
mask_imag, x_edges, y_edges = np.histogram2d(
    samps[0], samps[1], bins=[bins, bins], weights=samps[3], density=False
)
mask_real /= mask
mask_imag /= mask

dirty_img = np.abs(fftshift(ifft2(fftshift(mask_real + 1j * mask_imag))))

In [ ]:
fig, ax = plt.subplots(1, 2, layout="constrained", figsize=(9, 10))
im = ax[0].imshow(
    np.abs(mask_real + 1j * mask_imag), origin="lower", cmap="inferno", norm=LogNorm()
)
fig.colorbar(im, ax=ax[0], shrink=0.305, label="Intensity in a.u.")
ax[0].set_xlabel("pixels")
ax[0].set_ylabel("pixels")

im = ax[1].imshow(
    np.angle(mask_real + 1j * mask_imag),
    origin="lower",
    cmap="RdBu",
    norm=CenteredNorm(),
)
cbar = fig.colorbar(im, ax=ax[1], shrink=0.305, label="Phase in rad")
cbar.set_ticks(np.arange(-np.pi, 3 / 2 * np.pi, np.pi / 2))
cbar.set_ticklabels(["$-\\pi$", "$-\\pi/2$", "$0$", "$\\pi/2$", "$\\pi$"])

ax[1].set_xlabel("pixels")
ax[1].set_ylabel("pixels")

None

In [ ]:
fig, ax = plt.subplots(1, 2, layout="constrained", figsize=(9, 10))
im = ax[0].imshow(dirty_img, origin="lower", cmap="inferno")
fig.colorbar(im, ax=ax[0], shrink=0.29, label="Flux density in a.u.")
ax[0].set_xlabel("pixels")
ax[0].set_ylabel("pixels")

im = ax[1].imshow(dirty_img - img, origin="lower", cmap="RdBu", norm=CenteredNorm())
fig.colorbar(im, ax=ax[1], shrink=0.29, label="Difference in Flux density in a.u.")
ax[1].set_xlabel("pixels")
ax[1].set_ylabel("pixels")

None